<a href="https://colab.research.google.com/github/evinracher/3008410-intelligent-systems/blob/main/week2/exercise1/Fine_Tunning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%env TF_USE_LEGACY_KERAS=1

!pip -q uninstall -y keras transformers tensorflow tf-keras safetensors
!pip -q install \
  "tensorflow==2.20.0" \
  "tf-keras" \
  "transformers==4.49.0" \
  "safetensors==0.4.5" \
  "datasets==2.20.0" \
  "evaluate==0.4.2" \
  "accelerate==0.34.2" \
  "sentencepiece==0.2.0" rouge_score nltk

env: TF_USE_LEGACY_KERAS=1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM, create_optimizer, AdamWeightDecay, pipeline
from datasets import load_dataset
import tensorflow as tf
from datasets import Dataset
import evaluate
import numpy as np
import torch
import os

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
folder_path = "/content/"
dataset_name = "eng_small.csv"
path = os.path.join(folder_path, dataset_name)
print(path)
data = Dataset.from_csv(path, encoding='utf-8')
data = data.train_test_split(test_size=0.1)
print(data)

/content/eng_small.csv


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['engl', 'spa'],
        num_rows: 18096
    })
    test: Dataset({
        features: ['engl', 'spa'],
        num_rows: 2011
    })
})


In [8]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  #google/flan-t5-small
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small") #google/flan-t5-small
#model = export_and_get_onnx_model('t5-small')

prefix = "translate: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [9]:
tokenized_data = data.map(preprocess_function, batched=True, remove_columns=["engl", "spa"])

Map:   0%|          | 0/18096 [00:00<?, ? examples/s]

Map:   0%|          | 0/2011 [00:00<?, ? examples/s]

In [10]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")
optimizer = AdamWeightDecay(learning_rate=2e-4, weight_decay_rate=0.01) #2e-5 was before wd was 1e-2, Typically, 1e-4 and 3e-4 work well for most problems

In [11]:
# Convert to TF dataset using a workaround for numpy compatibility
tf_train_set = tokenized_data["train"].to_tf_dataset(
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_test_set = tokenized_data["test"].to_tf_dataset(
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [12]:
epochs = 5
model.compile(optimizer=optimizer)

In [13]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=epochs, callbacks=None)

Epoch 1/5
1131/1131 [==============================] - 293s 239ms/step - loss: 2.1541 - val_loss: 1.4120
Epoch 2/5
1131/1131 [==============================] - 236s 208ms/step - loss: 1.4959 - val_loss: 1.1304
Epoch 3/5
1131/1131 [==============================] - 284s 251ms/step - loss: 1.2229 - val_loss: 0.9892
Epoch 4/5
1131/1131 [==============================] - 277s 245ms/step - loss: 1.0555 - val_loss: 0.9067
Epoch 5/5
1131/1131 [==============================] - 310s 274ms/step - loss: 0.9376 - val_loss: 0.8496


In [14]:
# Guarda el modelo entrenado
folder_path = 'model'
model_name = "NMT-epocs-" + str(epochs)
path = os.path.join(folder_path, model_name + ".h5")
model.save_pretrained(path)
del model

In [15]:
#Para inferir desde aquí.
model_name = "NMT-epocs-" + str(epochs)
path = os.path.join(folder_path, model_name + ".h5")

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = TFAutoModelForSeq2SeqLM.from_pretrained(path, pad_token_id=tokenizer.eos_token_id)

summarizer = pipeline("translation",
    model=model,
    tokenizer=tokenizer,
    framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at model/NMT-epocs-5.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/__init__.py:1154: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(
Device set to use 0


In [16]:
import timeit
start_time = timeit.default_timer()

text = "translate: it's summer it is nice to go to the beach"
print(summarizer(text, min_length=4, max_length=100))

elapsed = timeit.default_timer() - start_time
print(f"time: {round(elapsed,2)} seconds")

[{'translation_text': 'Es el verano que irse a la boca.'}]
time: 10.77 seconds


# Exercise

- This time use the larger dataset (eng.csv), use the same sentence and see the results.
- Modify the code to graph and report the Rouge metric (*)
  

In [18]:
import evaluate
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

rouge = evaluate.load("rouge")

class RougeCallback(tf.keras.callbacks.Callback):
    def __init__(self, tokenizer, val_dataset, max_batches=None, gen_kwargs=None):
        """
        tokenizer: tokenizer HF
        val_dataset: tf.data.Dataset (tu tf_test_set)
        max_batches: si quieres evaluar solo una parte del val set (más rápido). None = todo.
        gen_kwargs: kwargs para model.generate (max_length, num_beams, etc.)
        """
        super().__init__()
        self.tokenizer = tokenizer
        self.val_dataset = val_dataset
        self.max_batches = max_batches
        self.gen_kwargs = gen_kwargs or {"max_length": 128}
        self.history = {"rouge1": [], "rouge2": [], "rougeL": [], "rougeLsum": []}

    def on_epoch_end(self, epoch, logs=None):
        preds_text = []
        refs_text = []

        for step, batch in enumerate(self.val_dataset):
            if self.max_batches is not None and step >= self.max_batches:
                break

            input_ids = batch["input_ids"]
            attention_mask = batch.get("attention_mask", None)
            labels = batch["labels"]

            # Generación (traducción)
            generated = self.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                **self.gen_kwargs
            )

            # Decode predicciones
            preds_text.extend(
                self.tokenizer.batch_decode(generated.numpy(), skip_special_tokens=True)
            )

            # Decode labels (reemplazar -100 por pad_token_id)
            labels_np = labels.numpy()
            labels_np = np.where(labels_np != -100, labels_np, self.tokenizer.pad_token_id)
            refs_text.extend(
                self.tokenizer.batch_decode(labels_np, skip_special_tokens=True)
            )

        scores = rouge.compute(predictions=preds_text, references=refs_text, use_stemmer=True)

        # Guardar en historial
        for k in self.history.keys():
            self.history[k].append(scores.get(k, 0.0))

        # Reportar
        msg = (
            f"\n[Epoch {epoch+1}] "
            f"ROUGE-1={scores.get('rouge1', 0.0):.4f} | "
            f"ROUGE-2={scores.get('rouge2', 0.0):.4f} | "
            f"ROUGE-L={scores.get('rougeL', 0.0):.4f} | "
            f"ROUGE-Lsum={scores.get('rougeLsum', 0.0):.4f}"
        )
        print(msg)

        # (Opcional) meter en logs para que quede en History de Keras
        if logs is not None:
            logs.update({k: float(v) for k, v in scores.items() if k in self.history})


# === USO ===
epochs = 5
model.compile(optimizer=optimizer)

rouge_cb = RougeCallback(
    tokenizer=tokenizer,
    val_dataset=tf_test_set,
    max_batches=20,                 # <-- ajusta: None para evaluar TODO, o 10/20 para ir rápido
    gen_kwargs={"max_length": 128}  # <-- puedes ajustar max_length / num_beams si quieres
)

history = model.fit(
    x=tf_train_set,
    validation_data=tf_test_set,
    epochs=epochs,
    callbacks=[rouge_cb]
)

# === GRAFICAR ROUGE ===
x = np.arange(1, epochs + 1)

plt.figure()
plt.plot(x, rouge_cb.history["rouge1"], label="ROUGE-1")
plt.plot(x, rouge_cb.history["rouge2"], label="ROUGE-2")
plt.plot(x, rouge_cb.history["rougeL"], label="ROUGE-L")
plt.plot(x, rouge_cb.history["rougeLsum"], label="ROUGE-Lsum")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("ROUGE on validation set by epoch")
plt.legend()
plt.show()

# === REPORTE FINAL (última época) ===
final_rouge = {k: v[-1] for k, v in rouge_cb.history.items()}
print("\nFinal ROUGE (last epoch):", {k: round(v, 4) for k, v in final_rouge.items()})


Epoch 1/5
1131/1131 [==============================] - ETA: 0s - loss: 0.6276
[Epoch 1] ROUGE-1=0.6396 | ROUGE-2=0.4354 | ROUGE-L=0.6344 | ROUGE-Lsum=0.6362
1131/1131 [==============================] - 364s 314ms/step - loss: 0.6276 - val_loss: 0.7365 - rouge1: 0.6396 - rouge2: 0.4354 - rougeL: 0.6344 - rougeLsum: 0.6362
Epoch 2/5
1131/1131 [==============================] - ETA: 0s - loss: 0.5856
[Epoch 2] ROUGE-1=0.6487 | ROUGE-2=0.4469 | ROUGE-L=0.6434 | ROUGE-Lsum=0.6440
1131/1131 [==============================] - 410s 363ms/step - loss: 0.5856 - val_loss: 0.7220 - rouge1: 0.6487 - rouge2: 0.4469 - rougeL: 0.6434 - rougeLsum: 0.6440
Epoch 3/5
1131/1131 [==============================] - ETA: 0s - loss: 0.5439

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:8 transformation with iterator: Iterator::Root::Prefetch::ParallelMapV2::Map: InvalidArgumentError: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:GPU:0}} dims must represent a vector, got shape [16,13] [Op:Fill]
Traceback (most recent call last):

  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/script_ops.py", line 145, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/script_ops.py", line 152, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "/u

If you want to try other tokenizers, see next

In [19]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [20]:
from transformers import AlbertTokenizer, AlbertModel

tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model = AlbertModel.from_pretrained("albert-base-v2")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

In [21]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import ElectraTokenizer, ElectraModel

tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
mymodel = ElectraModel.from_pretrained("google/electra-small-discriminator")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

### Exercise

- Change summarize by translate
- Graph the Rouge metric


In [24]:
import evaluate
import numpy as np
import matplotlib.pyplot as plt

rouge = evaluate.load("rouge")

def rouge_on_dataset(model, tokenizer, tf_dataset, max_batches=50, gen_kwargs=None):
    gen_kwargs = gen_kwargs or {"max_length": 128}

    preds_text, refs_text = [], []

    for step, batch in enumerate(tf_dataset):
        if max_batches is not None and step >= max_batches:
            break

        input_ids = batch["input_ids"]
        attention_mask = batch.get("attention_mask", None)
        labels = batch["labels"]

        generated = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **gen_kwargs
        )

        preds_text.extend(tokenizer.batch_decode(generated.numpy(), skip_special_tokens=True))

        labels_np = labels.numpy()
        labels_np = np.where(labels_np != -100, labels_np, tokenizer.pad_token_id)
        refs_text.extend(tokenizer.batch_decode(labels_np, skip_special_tokens=True))

    scores = rouge.compute(predictions=preds_text, references=refs_text, use_stemmer=True)
    keep = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    return {k: float(scores.get(k, 0.0)) for k in keep}

scores = rouge_on_dataset(
    model=model,
    tokenizer=tokenizer,
    tf_dataset=tf_test_set,
    max_batches=50,                 # None = todo el dataset (más lento)
    gen_kwargs={"max_length": 128}
)

print("ROUGE scores:", {k: round(v, 4) for k, v in scores.items()})

labels = list(scores.keys())
values = [scores[k] for k in labels]

plt.figure()
plt.bar(labels, values)
plt.ylim(0, 1)
plt.xlabel("ROUGE metric")
plt.ylabel("Score")
plt.title("ROUGE on test set")
plt.show()


TypeError: The current model class (RobertaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].